In [4]:
import os
import json
import pandas as pd
import fnmatch

# 시작 디렉토리를 현재 디렉토리로 설정
start_dir = '.'

# 결과를 저장할 빈 데이터프레임 생성
result_df = pd.DataFrame()

# 모든 하위 폴더 순회
for root, dirs, files in os.walk(start_dir):
    # .csv 파일들과 configWn.json 파일이 있는 폴더만 처리
    for file in files:
        if fnmatch.fnmatch(file, '*.csv') and any('configW' in f for f in files):
            # .csv 파일 불러오기
            data_df = pd.read_csv(os.path.join(root, file))
            
            # configWn.json 파일들 처리
            for i in range(len(data_df)):
                with open(os.path.join(root, f'configW{i+1}.json'), 'r') as f:
                    config_data = json.load(f)
                
                # config_data를 data_df의 행에 추가
                for key, value in config_data.items():
                    if isinstance(value, list) or isinstance(value, tuple):  # 값이 리스트 또는 튜플일 경우
                        value = ', '.join(map(str, value))  # 값들을 문자열로 변환하여 하나의 문자열로 합침
                    data_df.loc[i, key] = value
            
            # 결과 데이터프레임에 추가
            result_df = pd.concat([result_df, data_df], ignore_index=True)

# 결과 저장
result_df.to_csv('merge_result.csv', index=False)
